# CS 179 Project

In [100]:
import pyGM as gm
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
from pyGM.messagepass import LBP, NMF

# Load training data and reduce (subsample) if desired

# Read thru file to get numeric ids for each player 
with open('train.csv') as f: lines = f.read().split('\n')
with open('valid.csv') as f: lines2 = f.read().split('\n')

p = 0; playerid = {};
for i in range(len(lines)):
  csv = lines[i].split(',');
  if len(csv) != 10: continue;   # parse error or blank line
  player0,player1 = csv[1],csv[4];
  if player0 not in playerid: playerid[player0]=p; p+=1;
  if player1 not in playerid: playerid[player1]=p; p+=1;

nplayers = len(playerid)
playername = ['']*nplayers
for player in playerid: playername[ playerid[player] ]=player;  # id to name lookup

# Do we need this part?
# # Sparsifying parameters (discard some training examples):
# pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
# nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
# nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)

# nplays, nwins = np.zeros( (nplayers,nplayers) ), np.zeros( (nplayers,nplayers) );
# for i in range(len(lines)):
#   csv = lines[i].split(',');
#   if len(csv) != 10: continue;   # parse error or blank line
#   a,b = playerid[csv[1]],playerid[csv[4]];
#   aw,bw = csv[2]=='[winner]',csv[5]=='[winner]';
#   if (np.random.rand() < pKeep):
#     if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
#       nplays[a,b] += 1; nplays[b,a]+=1; nwins[a,b] += aw; nwins[b,a] += bw;

In [125]:
nlevels = 15
scale = 0.3

X = [None]*nplayers
for i in range(nplayers):
    X[i] = gm.Var(i, nlevels)

Pwin = np.zeros( (nlevels,nlevels) )
for i in range(nlevels):
    for j in range(nlevels):
        diff = i-j
        Pwin[i,j] = (1.0/(1+np.exp(-scale*diff)))

factors = [ gm.Factor([X[i]], 1.0/nlevels) for i in range(nplayers) ]
pKeep = 0.15  # keep a fraction of the 193k games

for i in range(len(lines)):
    csv = lines[i].split(',')
    if len(csv) != 10: continue
    P1,P2 = playerid[csv[1]],playerid[csv[4]]
    win = -1
    if csv[2]=='[winner]':
        win = 1
    if P1>P2: P1,P2,win=P2,P1,-win
    if P1 != P2 and np.random.rand() < pKeep:
        factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )

In [126]:
model = gm.GraphModel(factors)
model.makeMinimal()

In [127]:
# This one gives an error
#lnZ, bel = LBP(model, maxIter=10, verbose=True)

In [128]:
lnZ, bel = NMF(model, maxIter=5, verbose=True)

Iter 0: -29680.33984805207
Iter 1: -18465.954538794475
Iter 2: -18096.387694491677
Iter 3: -17994.821942682076
Iter 4: -17963.537384334693
Iter 5: -17951.25327788402


In [129]:
totalGames = 0
correctlyPredictedGames = 0

for i in range(len(lines2)):
    csv = lines2[i].split(',')
    if len(csv) != 10: continue
    P1,P2 = playerid[csv[1]],playerid[csv[4]]
    win = -1
    if csv[2]=='[winner]':
        win = 1
    if P1>P2: P1,P2,win=P2,P1,-win
    if P1 != P2:
        totalGames += 1
        predictedWin = (bel[P1]*bel[P2]*gm.Factor([X[P1],X[P2]],Pwin)).table.sum()
        if((predictedWin > 0.5 and win == 1) or (predictedWin < 0.5 and win == -1)):
            correctlyPredictedGames += 1

In [130]:
print(correctlyPredictedGames / totalGames)

0.6672800961630517
